In [1]:
import pandas as pd
import jellyfish
import numpy as np

In [2]:
def no_lastname(lname):
    return lname=='-' or lname=='0' if isinstance(lname,str) else True

In [3]:
def is_person(title):
    keywords = ['นาย','นาง','นางสาว']
    return title in keywords

In [4]:
#drop out spaces and .
def drop_separators(str):
    try:
        return str.replace('.','').replace(' ','')
    except:
        print('cannot process:',str)
        return ''

In [5]:
def drop_common_words(str):
    common_words = ['บริษัท','จำกัด','(ประเทศไทย)','(มหาชน)']
    out = str
    try:
        for word in common_words:
            out = out.replace(word, '')
    except:
        print('cannot process:',str)
        return ''
    return out

In [6]:
def preprocess_name(str):
    return drop_common_words(drop_separators(str)).strip()

In [7]:
#return the id's and the jaro scores for the top k matches of a firm name
#name = firm name to match. The name will be matched against the whole master file
#count = number of top matches
##############################
def get_top_matches(name, master, count=10):
    dist = master['mod_name'].apply(lambda a: jellyfish.jaro_distance(a,name))
    topk = dist.nlargest(count)
    indices = topk.index.values
    ids = master['jrst_id'][indices]
    return ids.values, topk.values

In [8]:
def match(query_filename, master_filename, output_prefix):
    try:
        query = pd.read_csv(query_filename)
    except:
        #thai
        query = pd.read_csv(query_filename, encoding='cp874')
        
    master = pd.read_csv(master_filename)

    #rename idx
    query = query.rename({'เลขทะเบียน':'id','คำนำหน้า':'title','ชื่อ':'name','สกุล':'lname','สัญชาติ':'nat','อาชีพ':'occ','จำนวนหุ้นที่ถือ':'stock'},axis='columns')

    #drop dup
    query = query.drop_duplicates()

    #dropping unused
    query = query.drop(['stock'],axis=1)

    #filter only th. nat
    query = query[query['nat'] == 'TH']

    #select only ones without last name
    query = query[query['lname'].apply(no_lastname)]
    
    #filter out using title
    query = query[~query['title'].apply(is_person)]

    master = master.drop_duplicates()

    #drop nan (last element is a nan)
    master = master.dropna()

    #drop unused column
    master = master.drop([
        'source'
    ],
    axis=1)

    #rename column
    master = master.rename({'jrst_nm':'name_2'},axis='columns')

    #define a temporary column containing modified firm names for matching purpose
    master['mod_name'] = master['name_2'].apply(preprocess_name)
    query['mod_name'] = query['name'].apply(preprocess_name)

    merged = pd.merge(query,master,on=['mod_name'],how='inner')

    #drop temporary column
    merged = merged.drop(['mod_name'],axis=1)

    #drop unused columns
    merged = merged.drop(['nat', 'occ', 'lname'],axis=1)

    #rename columns
    merged = merged.rename({'jrst_id':'id_in_master','name_2':'name_in_master'}, axis=1)

    #save
    merged.to_csv('{}_merged.csv'.format(output_prefix))

    #get unmatched names
    unmatched = query[~query['name'].isin(merged['name'])]

    #restart index number from 1
    unmatched = unmatched.reset_index(drop=True)

    #remove matched names
    #edit: Can't!? -- sometimes there are more than one flavors of the same name where one perfectly matches and others don't.
    #master = master[~master['name_2'].isin(merged['name'])]

    #create columns for similarity
    unmatched['jrst_id'], unmatched['sim_score'] = 0,0.0

    #indices for the newly created columns
    idloc = unmatched.columns.get_loc('jrst_id')
    scoreloc = unmatched.columns.get_loc('sim_score')

    #create empty placeholder for matched data
    partial_match = pd.DataFrame(columns=unmatched.columns)

    size = unmatched.shape[0]
    for i in range(unmatched.shape[0]):
        firm_name = unmatched['mod_name'][i]
        #get top k matches
        ids, scores = get_top_matches(firm_name, master)
        print("{} of {}".format(i, size))
        #add all top k matches to the placeholder
        for j in range(len(ids)):
            #first, modify the original query set
            unmatched.iat[i,idloc] = ids[j]
            unmatched.iat[i,scoreloc] = scores[j]
            #copy the original row to the new dataframe
            partial_match = partial_match.append(unmatched.loc[i])

    partial_match = pd.merge(partial_match, master[['jrst_id','name_2']],on=['jrst_id'],how='left')

    partial_match = partial_match.drop(['nat', 'occ', 'lname', 'mod_name'], axis=1)
    partial_match = partial_match.rename({'jrst_id':'id_in_master','name_2':'name_in_master'}, axis=1)


    partial_match.to_csv('{}_partial.csv'.format(output_prefix))

In [9]:
import os
output_path = './output'
input_path = './queryfiles'

In [10]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [11]:
files = os.listdir(input_path)
files = list(filter(lambda s: '.csv' in s, files))

In [12]:
master_filename = './master.csv'

idx = 0
for file in files: 
    print(input_path+'/'+file)
    idx = idx + 1
    
    print('===============================================')
    print('Processing file {} of {}'.format(idx,len(files)))
    print('===============================================')

    query_filename = file
    prefix = file[:-4]
    match(input_path+'/'+query_filename, master_filename, output_path+'/'+prefix)

./queryfiles/TS_10-60-108121_009.csv
Processing file 1 of 18
0 of 140
1 of 140
2 of 140
3 of 140
4 of 140
5 of 140
6 of 140
7 of 140
8 of 140
9 of 140
10 of 140
11 of 140
12 of 140
13 of 140
14 of 140
15 of 140
16 of 140
17 of 140
18 of 140
19 of 140
20 of 140
21 of 140
22 of 140
23 of 140
24 of 140
25 of 140
26 of 140
27 of 140
28 of 140
29 of 140
30 of 140
31 of 140
32 of 140
33 of 140
34 of 140
35 of 140
36 of 140
37 of 140
38 of 140
39 of 140
40 of 140
41 of 140
42 of 140
43 of 140
44 of 140
45 of 140
46 of 140
47 of 140
48 of 140
49 of 140
50 of 140
51 of 140
52 of 140
53 of 140
54 of 140
55 of 140
56 of 140
57 of 140
58 of 140
59 of 140
60 of 140
61 of 140
62 of 140
63 of 140
64 of 140
65 of 140
66 of 140
67 of 140
68 of 140
69 of 140
70 of 140
71 of 140
72 of 140
73 of 140
74 of 140
75 of 140
76 of 140
77 of 140
78 of 140
79 of 140
80 of 140
81 of 140
82 of 140
83 of 140
84 of 140
85 of 140
86 of 140
87 of 140
88 of 140
89 of 140
90 of 140
91 of 140
92 of 140
93 of 140
94 of 140